
# Arguments

In [1]:
#load from last improved and do 1 epoch only


batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

model_save_path = 'model-warmup.h5'

checkpoint_path = 'checkpoints-warmup.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
num_of_epochs = 30
start_epoch = 0
# --------------------------------------------------

# Print class names

In [2]:
# Get classes
import os
import re

class_names = []
class_paths_plant_village = []
class_paths_plant_self = []
class_paths_plant_validation = []

for path in dataset_paths:
    classes = os.listdir(path)
    for i in classes:
        if(re.search('___', i)):
            if(i not in class_names):
                class_names.append(i)
            if(re.search('plantvillage', path)):
                class_paths_plant_village.append(path + '/' + i)
            elif(re.search('self', path)):
                class_paths_plant_self.append(path + '/' + i)
            else:
                class_paths_plant_validation.append(path + '/' + i)

print(class_names)   
print()
print('PlantVillage')
print(class_paths_plant_village) 
print(len(class_paths_plant_village))
print()
print('Self')
print(class_paths_plant_self) 
print(len(class_paths_plant_self))
print()
print('Validation')
print(class_paths_plant_validation) 
print(len(class_paths_plant_validation))

['___Yellow_Leaf_Curl_Virus', '___Late_blight', '___Early_blight', '___Two-spotted_spider_mite', '___Septoria_leaf_spot', '___Bacterial_spot', '___healthy', '___Target_Spot', '___Mosaic_Virus', '___Leaf_Mold', '___Leaf_miner', '___Curly_top_virus', '___Appids']

PlantVillage
['../datasets/plantvillage/___Yellow_Leaf_Curl_Virus', '../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Early_blight', '../datasets/plantvillage/___Two-spotted_spider_mite', '../datasets/plantvillage/___Septoria_leaf_spot', '../datasets/plantvillage/___Bacterial_spot', '../datasets/plantvillage/___healthy', '../datasets/plantvillage/___Target_Spot', '../datasets/plantvillage/___Mosaic_Virus', '../datasets/plantvillage/___Leaf_Mold']
10

Self
['../datasets/self/___Leaf_miner', '../datasets/self/___Early_blight', '../datasets/self/___Curly_top_virus', '../datasets/self/___Appids']
4

Validation
['../datasets/validation/___Leaf_miner', '../datasets/validation/___Yellow_Leaf_Curl_Virus', '../data

# Select training classes

early blight in plantVillage dataset replaced by self one

In [3]:
class_paths_training = ['../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner', '../datasets/self/___Curly_top_virus']
class_names = None

# Load dataset

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.dataset_loader import DatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = resizeImagePreprocessor(input_width, input_height)
    iap= ImgToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    dl = DatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = dl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/4401
[INFO]: Processed 1000/4401
[INFO]: Processed 1500/4401
[INFO]: Processed 2000/4401
[INFO]: Processed 2500/4401
[INFO]: Processed 3000/4401
[INFO]: Processed 3500/4401
[INFO]: Processed 4000/4401


In [5]:
print('Dataset loaded')
print(class_names)

Dataset loaded
['___Appids', '___Curly_top_virus', '___Early_blight', '___Leaf_miner']



# Load model

In [6]:
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
from keras.layers import Input

base_model = VGG16(weights='imagenet',include_top=False,
                  input_tensor=Input(shape = (input_width,input_height, input_depth)))

# plot_save_path = 'diagram-base-resnet50.png'
# plot_model(base_model, to_file=plot_save_path, show_shapes=True)

# Create custom head

In [7]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

head_model = base_model.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256,activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
# Add a softmaxc layer
head_model = Dense(len(class_names),activation='softmax')(head_model)

In [8]:
from keras.models import Model


# place the head FC model on top of the base model 
model = Model(inputs=base_model.input, outputs = head_model)

# plot_save_path = 'diagram-vgg16-head-attached.png'
# plot_model(model, to_file=model_diagram_path, show_shapes=True)

# Add dropout layers

In [9]:
from keras.layers import Dropout
from keras.models import Model

layers = [l for l in model.layers]


print(layers[0].name)
print(len(layers))

x = layers[0].output

# Add until layer layer 15
for i in range(1, 17):
    x = layers[i](x)

# Add the two dropout layers
x = Dropout(0.5)(x)
x = layers[17](x)
x = Dropout(0.5)(x)

# Add the rest of the layers
for i in range(18, len(layers)):
    x = layers[i](x)

model = Model(input=layers[0].input, output=x)

# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {:5}\t{:30}{}".format(i, layer.name, layer.__class__.__name__))

input_1
23
[INFO]     0	input_1                       InputLayer
[INFO]     1	block1_conv1                  Conv2D
[INFO]     2	block1_conv2                  Conv2D
[INFO]     3	block1_pool                   MaxPooling2D
[INFO]     4	block2_conv1                  Conv2D
[INFO]     5	block2_conv2                  Conv2D
[INFO]     6	block2_pool                   MaxPooling2D
[INFO]     7	block3_conv1                  Conv2D
[INFO]     8	block3_conv2                  Conv2D
[INFO]     9	block3_conv3                  Conv2D
[INFO]    10	block3_pool                   MaxPooling2D
[INFO]    11	block4_conv1                  Conv2D
[INFO]    12	block4_conv2                  Conv2D
[INFO]    13	block4_conv3                  Conv2D
[INFO]    14	block4_pool                   MaxPooling2D
[INFO]    15	block5_conv1                  Conv2D
[INFO]    16	block5_conv2                  Conv2D
[INFO]    17	dropout_2                     Dropout
[INFO]    18	block5_conv3                  Conv2D
[INFO]    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


# Freeze base model

In [10]:
# traverse all layers and freeze the weight of the corresponding layer
for layer in base_model.layers:
    layer.trainable = False

In [11]:
from keras.utils.vis_utils import plot_model

# plot_save_path = 'diagram-vgg16-head-attached.png'
# plot_model(model, to_file=plot_save_path, show_shapes=True)

# Compile model

In [12]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# Since we only train the new fully connected layer, we do a few iterations

In [13]:
print(model.metrics_names)

['loss', 'accuracy']


# Checkpoints

In [14]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [15]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training monitor

In [16]:
from utils.callbacks.monitor import Monitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(Monitor(fig_path, json_path, start_epoch))

# Warm up head

In [17]:
model.fit_generator(
    aug.flow(train_x,train_y, batch_size = batch_size),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

model.save(model_save_path)

Epoch 1/30
103/103 [==============================] - 59s 575ms/step - loss: 3.2291 - accuracy: 0.4715 - val_loss: 0.9555 - val_accuracy: 0.6067

Epoch 00001: val_loss improved from inf to 0.95555, saving model to checkpoints-warmup.hdf5
Epoch 2/30
103/103 [==============================] - 54s 520ms/step - loss: 0.9124 - accuracy: 0.6077 - val_loss: 0.6342 - val_accuracy: 0.8047

Epoch 00002: val_loss improved from 0.95555 to 0.63421, saving model to checkpoints-warmup.hdf5
Epoch 3/30
103/103 [==============================] - 54s 521ms/step - loss: 0.7835 - accuracy: 0.6784 - val_loss: 0.5160 - val_accuracy: 0.8238

Epoch 00003: val_loss improved from 0.63421 to 0.51599, saving model to checkpoints-warmup.hdf5
Epoch 4/30
103/103 [==============================] - 53s 512ms/step - loss: 0.7106 - accuracy: 0.7209 - val_loss: 0.5179 - val_accuracy: 0.8229

Epoch 00004: val_loss did not improve from 0.51599
Epoch 5/30
103/103 [==============================] - 53s 510ms/step - loss: 0.64

# Evaluate with 20% test set from same dataset

In [18]:
from sklearn.metrics import classification_report

print("[INFO] evaluating with test set...")
predictions = model.predict(test_x,batch_size=batch_size)
print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=class_names))

[INFO] evaluating with test set...
                    precision    recall  f1-score   support

         ___Appids       0.92      0.96      0.94       243
___Curly_top_virus       0.67      0.90      0.77       109
   ___Early_blight       0.98      0.84      0.90       388
     ___Leaf_miner       0.91      0.94      0.92       361

       avg / total       0.91      0.90      0.90      1101



# Evaluate with validation images

In [19]:
class_paths_validation = ['../datasets/validation/___Early_blight', '../datasets/validation/___Appids', '../datasets/validation/___Leaf_miner', '../datasets/self/___Curly_top_virus']

In [20]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] loading images...
[INFO]: Processed 500/610
[INFO] evaluating with validation set...
                    precision    recall  f1-score   support

         ___Appids     0.5385    0.7000    0.6087        10
___Curly_top_virus     0.9565    0.9402    0.9483       117
   ___Early_blight     0.8182    0.7500    0.7826        12
     ___Leaf_miner     0.7143    0.7143    0.7143        14

       avg / total     0.8962    0.8889    0.8917       153

